In [3]:
import os
import polars as pl

In [4]:
df = pl.read_parquet(os.path.join("data","fdb.parquet"))

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [5]:
df.select(pl.col("datum").min())

datum
datetime[μs]
2007-01-01 10:05:00


In [6]:
df.filter(pl.col("datum").dt.month() == 12)

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""Prima LOVE""",2012-12-21 12:50:00,"""Nouzové přistání""","""18.04.2017 v 12:45 (ORF1)19.04…","""Další díl/opakování:18.04.2017…","""serial/nouzove-pristani-hot-in…"
"""Film+""",2011-12-09 19:10:00,"""Jak nezískat Pulitzera""",null,null,"""film/jak-neziskat-pulitzera-sh…"
"""Cinemax""",2014-12-26 08:05:00,"""Velká krize""","""13.12.2019 v 04:20 (HBO3)14.12…","""Další díl/opakování:13.12.2019…","""film/velka-krize-too-big-to-fa…"
"""Nova Action""",2018-12-12 21:10:00,"""Kriminálka Las Vegas""","""15.04.2025 v 16:00 (Paramount …","""Další díl/opakování:15.04.2025…","""serial/kriminalka-las-vegas-cs…"
"""ČT2""",2022-12-25 23:55:00,"""Země - Den na zázračné planetě""",null,"""Dokument. Během jediného dne n…",null
…,…,…,…,…,…
"""Prima""",2007-12-18 08:40:00,"""Věřte - nevěřte""","""02.01.2022 v 05:50 (RTL2)08.01…","""Další díl/opakování:02.01.2022…","""serial/verte-neverte-beyond-be…"
"""CS Film""",2024-12-26 07:55:00,"""Já Baryk""","""20.01.2025 v 08:05 (CS Film)21…","""Další díl/opakování:20.01.2025…","""serial/ja-baryk/35440"""
"""Prima LOVE""",2012-12-04 17:30:00,"""Sabrina - mladá čarodějnice""","""06.02.2018 v 15:55 (Relax)07.0…","""Další díl/opakování:06.02.2018…","""serial/sabrina-mlada-carodejni…"


In [7]:
def kolikateho(porad):
    return pl.DataFrame({'den':[x for x in range(1,32)]}).with_columns(pl.col('den').cast(pl.Int8)).join(
        df.filter(pl.col("datum").dt.month() == 12).filter(pl.col("nazev").str.starts_with(porad)).with_columns(pl.col("datum").dt.day().alias("den").cast(pl.Int8)).group_by("den").len().with_columns(pl.lit(porad).alias("co")),
        how='left',
        on='den'
    ).fill_null(0)

In [8]:
pl.Config(tbl_rows=100)

In [9]:
df.select(pl.col("datum").min())

datum
datetime[μs]
2007-01-01 10:05:00


In [10]:
df.select(pl.col("datum").max())

datum
datetime[μs]
2025-04-17 23:55:00


In [11]:
df.filter(pl.col("nazev").str.starts_with("Císařův"))

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2023-04-09 15:55:00,"""Císařův pekař""","""21.04.2025 v 16:20 (ČT1)""","""Další díl/opakování:21.04.2025…","""film/cisaruv-pekar-a-pekaruv-c…"
"""Nova Cinema""",2020-12-30 13:00:00,"""Císařův pekař""",null,"""Komedie ČR (1951). Stárnoucí c…","""film/cisaruv-pekar-a-pekaruv-c…"
"""NOVA""",2023-12-29 15:45:00,"""Císařův pekař""","""21.04.2025 v 16:20 (ČT1)""","""Další díl/opakování:21.04.2025…","""film/cisaruv-pekar-a-pekaruv-c…"
"""CS Film""",2019-12-30 12:00:00,"""Císařův pekař, Pekařův císař""",null,"""Komedie ČR (1951). Film se vyz…","""film/cisaruv-pekar-a-pekaruv-c…"
"""CS Film""",2018-06-27 20:00:00,"""Císařův pekař, Pekařův císař""",null,"""Komedie ČR (1951). Film se vyz…","""film/cisaruv-pekar-a-pekaruv-c…"
"""CS Film""",2024-06-26 20:00:00,"""Císařův pekař, Pekařův císař""",null,"""Komedie ČR (1951). Film se vyz…","""film/cisaruv-pekar-a-pekaruv-c…"
"""Nova Cinema""",2018-05-15 18:25:00,"""Císařův pekař""",null,"""Komedie ČR (1951). Stárnoucí c…","""film/cisaruv-pekar-a-pekaruv-c…"
"""ČT1""",2013-12-28 09:20:00,"""Císařův pekař""","""21.04.2025 v 16:20 (ČT1)""","""Další díl/opakování:21.04.2025…","""film/cisaruv-pekar-a-pekaruv-c…"
"""ČT1""",2011-12-28 10:25:00,"""Císařův pekař a pekařův císař …","""21.04.2025 v 16:20 (ČT1)""","""Další díl/opakování:21.04.2025…","""film/cisaruv-pekar-a-pekaruv-c…"


In [12]:
df.filter(pl.col('url').str.contains('film/')).filter(pl.col("datum").dt.day() == 30).filter(pl.col("datum").dt.month() == 12).group_by("nazev").agg(pl.col("datum").dt.year().unique().len()).sort(by="datum",descending=True).head(100)

nazev,datum
str,u32
"""Události""",16
"""Pod pokličkou""",12
"""Branky, body, vteřiny""",12
"""Televizní noviny, Sportovní no…",12
"""Krimi zprávy""",11
"""Chalupa je hra""",10
"""Prostřeno!""",9
"""Události v regionech""",9
"""Nikdo není dokonalý""",9


In [13]:
filmy = list(set([
    "Anděl na horách",
    "Staré pověsti české",
    "Adéla ještě nevečeřela",
    "Manéž Bolka Polívky",
    "Anděl Páně",
    "Urbi et Orbi",
    "Černí baroni",
    "Grinch",
    "Vesničko má středisková",
    "Co takhle svatba, princi?",
    "Jak se budí princezny",
    "Dívka na koštěti",
    "Noc na Karlštejně",
    "Veselé Vánoce přejí chobotnice",
    "Shrek",
    "Z pekla štěstí",
    "Princezna ze mlejna",
    "Mr. Bean",
    "Anton Špelec, ostrostřelec",
    "Tootsie",
    "Lotrando a Zubejda",
    "Dovolená s Andělem",
    "Světáci",
    "Andělská tvář",
    "Pelíšky",
    "S čerty nejsou žerty",
    "Princ a večernice",
    "Bohouš",
    "Silvestry Vladimíra Menšíka",
    "Tři vejce do skla",
    "Šíleně smutná princezna",
    "Pyšná princezna",
    "Tři oříšky pro Popelku",
    "Princezna se zlatou hvězdou",
    "Dařbuján a Pandrhola",
    "Byl jednou jeden král",
    "Z pekla štěstí",
    "Nesmrtelná teta",
    "Sám doma",
    "Tři veteráni",
    "Popelka",
    "Peklo s princeznou",
    "Tři bratři",
    "Honza málem králem",
    "Zlatovláska",
    "O princezně Jasněnce a létajícím ševci",
    "Dobrý voják Švejk",
    "Chobotnice z II. patra",
    "Pretty Woman",
    "Císařův pekař",
    "Česká soda",
    "Mrazík",
    "Tři veteráni",
    "Jak vytrhnout velrybě stoličku",
    "Prázdniny v Římě",
    "Láska nebeská",
    "Notting Hill"
]))

do_grafu = pl.concat([kolikateho(film) for film in filmy])

In [14]:
import altair as alt

In [15]:
alt.Chart(
    do_grafu
).mark_rect().encode(
    alt.X('den:N', axis=alt.Axis(title=None)),
    alt.Y('co:N', axis=alt.Axis(title=None)),
    alt.Color('len:Q', legend=None)
)

alt.Chart(...)

In [16]:
def kolikateho_full(slovo):
    return pl.DataFrame({'den':[x for x in range(1,32)]}).with_columns(pl.col('den').cast(pl.Int8)).join(
        df.filter(pl.col("datum").dt.month() == 12).filter(pl.col("extra").str.contains(slovo) | pl.col("noticka").str.contains(slovo)).with_columns(pl.col("datum").dt.day().alias("den").cast(pl.Int8)).group_by("den").len().with_columns(pl.lit(slovo).alias("co")),
        how='left',
        on='den'
    ).fill_null(0)

In [17]:
herci = list(set([
    "Burian",
    "Menšík",
    "Donutil",
    "Polívk",
    "Izer",
    "Bohdalov",
    "Kemr",
    "Svěrák",
    "Hitler",
    "Stalin",
    "Janžurov",
    "Ježíš",
    "Brodský",
    "Jiráskov[áéo]",
    "Abrhám",
    "Šafránkov",
    "Smoljak",
    "Culkin",
    "Willis",
    "Grant",
    "Hanks",
    "Murray",
    "Roberts"
]))

do_grafu2 = pl.concat([kolikateho_full(herec) for herec in herci]).filter(pl.col("co").is_in(herci))

In [18]:
alt.Chart(
    do_grafu2, width=300,
    title="Kdo bývá v prosinci v televizi a kdy"
).mark_rect().encode(
    alt.X('den:N', axis=alt.Axis(title=None)),
    alt.Y('co:N', axis=alt.Axis(title=None), sort=sorted([h for h in herci if h is not None])),
    alt.Color('len:Q', legend=None)
)

alt.Chart(...)

In [19]:
df.columns

['kanal', 'datum', 'nazev', 'extra', 'noticka', 'url']

## 11. 11. 2025

In [21]:
df.select(pl.col('kanal')).unique()

kanal
cat
"""Nova Lady"""
"""CS Film"""
"""ČT2"""
"""Prima Show"""
"""Film Europe Channel"""
"""Nova Action"""
"""Cinemax"""
"""Prima"""
"""Cinemax2"""


In [22]:
kanaly_chceme = ['ČT1','ČT2','Prima','NOVA']

In [23]:
ruzne_roky = df.filter(pl.col('kanal').is_in(kanaly_chceme)).group_by('nazev').agg(pl.col('datum').dt.year().unique().len()).filter(pl.col('datum') >= 10).select(pl.col('nazev')).to_series().to_list()

In [88]:
df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
).filter(
    pl.col('nazev').str.contains('Pelíš')
)

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Pelíšky""",14,8,1.75


In [25]:
df.filter(pl.col("nazev") == "Pelíšky")

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2024-08-20 20:10:00,"""Pelíšky""",null,"""Komedie ČR (1999). Jedna z nej…","""film/pelisky/25517"""
"""ČT1""",2020-12-24 21:55:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2016-12-24 21:45:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2019-06-09 20:00:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2013-12-24 20:30:00,"""Pelíšky""",null,"""Film ČR (1999). Jedna z nejobl…","""film/pelisky/25517"""
"""ČT1""",2019-12-24 22:05:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2015-08-02 20:00:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna …","""film/pelisky/25517"""
"""ČT1""",2023-12-24 22:00:00,"""Pelíšky""",null,"""Komedie ČR (1999). Jedna z nej…","""film/pelisky/25517"""
"""ČT1""",2016-07-17 20:00:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""


In [92]:
df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
).head(150)

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Novoroční přípitek""",40,0,inf
"""Vánoční posezení u muziky""",28,0,inf
"""Silvestrovské Televizní noviny""",18,0,inf
"""U muziky na Silvestra""",17,0,inf
"""Čepice""",16,0,inf
"""Silvestry Vladimíra Menšíka""",16,0,inf
"""Štědrý večer pana rady Vacátka""",15,0,inf
"""Vánoce s Hutkou""",14,0,inf
"""Anděl Páně""",14,0,inf


In [94]:
vyber_predbezny = df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
)

In [96]:
vyber_predbezny.filter(pl.col("nazev").str.contains("Četník"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64


In [98]:
vyber_predbezny.filter(pl.col("nazev").str.contains("tanec"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64


In [102]:
vyber_predbezny.filter(pl.col("nazev").str.contains("Nesmrtelná t"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Nesmrtelná teta""",17,79,0.21519


In [118]:
vyber_vypocitany = df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    (pl.col('prosinec') <= 60) & (pl.col('pomer') >= 1)
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
)

In [120]:
print(vyber_vypocitany)

shape: (71, 4)
┌─────────────────────────────────┬──────────┬───────────────┬──────────┐
│ nazev                           ┆ prosinec ┆ unor-listopad ┆ pomer    │
│ ---                             ┆ ---      ┆ ---           ┆ ---      │
│ str                             ┆ u32      ┆ u32           ┆ f64      │
╞═════════════════════════════════╪══════════╪═══════════════╪══════════╡
│ Novoroční přípitek              ┆ 40       ┆ 0             ┆ inf      │
│ Vánoční posezení u muziky       ┆ 28       ┆ 0             ┆ inf      │
│ Silvestrovské Televizní noviny  ┆ 18       ┆ 0             ┆ inf      │
│ U muziky na Silvestra           ┆ 17       ┆ 0             ┆ inf      │
│ Čepice                          ┆ 16       ┆ 0             ┆ inf      │
│ Silvestry Vladimíra Menšíka     ┆ 16       ┆ 0             ┆ inf      │
│ Štědrý večer pana rady Vacátka  ┆ 15       ┆ 0             ┆ inf      │
│ Silvestr svobodného pána        ┆ 14       ┆ 0             ┆ inf      │
│ Mississippi Queen z P

In [122]:
vyber_vypocitany = vyber_vypocitany.select(
    pl.col("nazev")
).to_series(
).to_list(
)

In [124]:
vyber_vypocitany

['Novoroční přípitek',
 'Vánoční posezení u muziky',
 'Silvestrovské Televizní noviny',
 'U muziky na Silvestra',
 'Čepice',
 'Silvestry Vladimíra Menšíka',
 'Štědrý večer pana rady Vacátka',
 'Silvestr svobodného pána',
 'Mississippi Queen z Pohořelic',
 'Vánoce s Hutkou',
 'Ráno budeme moudřejší',
 'Anděl Páně',
 'Když čerti nosí štěstí',
 'Ten vánoční čas',
 'Prázdniny v Římě',
 'To vánoční šturmování aneb Pokoj lidem dobré vůle',
 'Do roka a do dna',
 'Česká mše vánoční v Římě',
 'Poklad',
 'Tajemství mořské panny',
 'Dr. Dolittle 2',
 'Někdo to rád horké',
 'O kominickém učni a dceři cukráře',
 'Vínečko, ne voda',
 'Vánoce',
 'Novoroční koncert Vídeňských filharmonik...',
 'Co takhle svatba, princi?',
 'Ať přiletí čáp, královno!',
 'Zas je tady Silvestr',
 'Rolničky, kam se podíváš',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Princové jsou na draka',
 'Z pekla štěstí 2',
 'Kotva u přívozu',
 'Dr. Dolittle',
 'Silvestr s četníky',
 'Tři oříšky pro Popelku',
 'Dunston: Sám v 

In [126]:
vyber_vypocitany.sort()

In [128]:
vyber_vypocitany

['Anděl Páně',
 'Ať přiletí čáp, královno!',
 'Bohouš',
 'Byl jednou jeden král...',
 'Cesta do Ameriky',
 'Chobotnice z II. patra',
 'Co takhle svatba, princi?',
 'Císař a tambor',
 'Císařův pekař',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Do roka a do dna',
 'Dr. Dolittle',
 'Dr. Dolittle 2',
 'Duch nad zlato',
 'Dunston: Sám v hotelu',
 'Hana a Petr Ulrychovi - Javorová píseň',
 'Jak dostat tatínka do polepšovny',
 'Jak vytrhnout velrybě stoličku',
 'Když čerti nosí štěstí',
 'Kotva u přívozu',
 'Lotrando a Zubejda',
 'Láska nebeská',
 'Láska rohatá',
 'Mississippi Queen z Pohořelic',
 'Mrazík',
 'Muž se železnou maskou',
 'Nebojsa',
 'Novoroční koncert Vídeňských filharmonik...',
 'Novoroční přípitek',
 'Někdo to rád horké',
 'O kominickém učni a dceři cukráře',
 'O medvědu Ondřejovi',
 'Pelíšky',
 'Poklad',
 'Pretty Woman',
 'Princové jsou na draka',
 'Prázdniny v Římě',
 'Pták Ohnivák',
 'Rolničky, kam se podíváš',
 'RumplCimprCampr',
 'Ráno budeme moudřejší',
 'S čerty 

In [35]:
vanocni_vyber = [
    "Prázdniny v Římě",
    "Anděl Páně",
    "Silvestr svobodného pána",
    "Šťastný smolař",
    "Dr. Dolittle 2",
    "Někdo to rád horké",
    "Mrazík",
    "Ať přiletí čáp, královno!",
    "Co takhle svatba, princi?",
    "Císařův pekař",
    "Kotva u přívozu",
    "Cesta do Rokycan",
    "Z pekla štěstí 2",
    "Byl jednou jeden král...",
    "Tři oříšky pro Popelku",
    "Dr. Dolittle",
    "Shrek 2",
    "Pekařův císař",
    "Láska nebeská",
    'Dalskabáty, hříšná ves aneb Zapomenutý čert',
    'Jak se budí princezny',
    'Sám doma 2: Ztracen v New Yorku',
    'Sám doma',
    'Princové jsou na draka',
    'Bohouš',
    'Nebožtíci přejí lásce',
    'Honza málem králem',
    'S čerty nejsou žerty',
    'O medvědu Ondřejovi',
    'Princezna ze mlejna',
    'Kráska a zvíře',
    'Hrátky s čertem',
    'Z pekla štěstí',
    'S tebou mě baví svět',
    'Princ a Večernice',
    'Dunston: Sám v hotelu',
    'Bejvalek se nezbavíš',
    'Veselé Vánoce přejí chobotnice',
    'Shrek',
    'Láska rohatá',
    'Grinch',
    'Světáci'
]

In [36]:
vanocni_vyber.sort()

In [37]:
vanocni_vyber

['Anděl Páně',
 'Ať přiletí čáp, královno!',
 'Bejvalek se nezbavíš',
 'Bohouš',
 'Byl jednou jeden král...',
 'Cesta do Rokycan',
 'Co takhle svatba, princi?',
 'Císařův pekař',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Dr. Dolittle',
 'Dr. Dolittle 2',
 'Dunston: Sám v hotelu',
 'Grinch',
 'Honza málem králem',
 'Hrátky s čertem',
 'Jak se budí princezny',
 'Kotva u přívozu',
 'Kráska a zvíře',
 'Láska nebeská',
 'Láska rohatá',
 'Mrazík',
 'Nebožtíci přejí lásce',
 'Někdo to rád horké',
 'O medvědu Ondřejovi',
 'Pekařův císař',
 'Princ a Večernice',
 'Princezna ze mlejna',
 'Princové jsou na draka',
 'Prázdniny v Římě',
 'S tebou mě baví svět',
 'S čerty nejsou žerty',
 'Shrek',
 'Shrek 2',
 'Silvestr svobodného pána',
 'Světáci',
 'Sám doma',
 'Sám doma 2: Ztracen v New Yorku',
 'Tři oříšky pro Popelku',
 'Veselé Vánoce přejí chobotnice',
 'Z pekla štěstí',
 'Z pekla štěstí 2',
 'Šťastný smolař']

In [38]:
vyber_vypocitany.sort()

In [39]:
vyber_vypocitany = [x for x in vyber_vypocitany if x != "Kráska a zvíře"]

In [40]:
with open("data_raw/vanocni_vyber_vypocitany.json", "w+", encoding="utf-8") as exp:
    exp.write(json.dumps(vyber_vypocitany))

In [41]:
len(vanocni_vyber)

42

Co do dat (přes Wikidata q):
- země původu
- kino x televizní
- rok výroby
- nejčastější stanice?
- žánr?
- hrají
- scénář

In [43]:
df.filter(pl.col("nazev").str.contains("Otázky Václava Moravce")).sample(5)

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2017-09-24 12:00:00,"""Otázky Václava Moravce""","""06.04.2025 v 12:00 (ČT24)06.04…","""Další díl/opakování:06.04.2025…","""film/otazky-vaclava-moravce/73…"
"""ČT1""",2008-04-06 12:00:00,"""Otázky Václava Moravce""","""06.04.2025 v 12:00 (ČT24)06.04…","""Další díl/opakování:06.04.2025…","""film/otazky-vaclava-moravce/73…"
"""ČT1""",2010-07-25 12:00:00,"""Otázky Václava Moravce""","""06.04.2025 v 12:00 (ČT24)06.04…","""Další díl/opakování:06.04.2025…","""film/otazky-vaclava-moravce/73…"
"""ČT1""",2010-02-22 04:00:00,"""Otázky Václava Moravce 2. část""","""06.04.2025 v 12:00 (ČT24)06.04…","""Další díl/opakování:06.04.2025…","""film/otazky-vaclava-moravce/73…"
"""ČT1""",2020-10-04 12:00:00,"""Otázky Václava Moravce""","""06.04.2025 v 12:00 (ČT24)06.04…","""Další díl/opakování:06.04.2025…","""film/otazky-vaclava-moravce/73…"


In [44]:
df.filter(pl.col("nazev").str.contains("Tři oříšky pro Popelku"))

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""VOYO""",2019-06-26 16:55:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR/N (1973). Krásnou, …","""film/tri-orisky-pro-popelku/21…"
"""Prima""",2014-12-24 20:15:00,"""Tři oříšky pro Popelku""","""Pohádka ČR (1973). O chudé Pop…",null,"""film/tri-orisky-pro-popelku/21…"
"""VOYO""",2013-07-10 13:05:00,"""Tři oříšky pro Popelku""",null,"""Film ČR/N (1973). Krásnou, ale…","""film/tri-orisky-pro-popelku/21…"
"""VOYO""",2015-12-25 20:00:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR/N (1973). Krásnou, …","""film/tri-orisky-pro-popelku/21…"
"""NOVA""",2011-06-24 20:00:00,"""Tři oříšky pro Popelku""",null,null,"""film/tri-orisky-pro-popelku/21…"
"""ČT1""",2019-12-28 15:35:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR/N (1973). Klasická …","""film/tri-orisky-pro-popelku/21…"
"""Nova Cinema""",2022-12-28 16:45:00,"""Tři oříšky pro Popelku""",null,"""Pohádka ČR/N (1973). Krásnou, …","""film/tri-orisky-pro-popelku/21…"
"""NOVA""",2009-12-24 20:00:00,"""Tři oříšky pro Popelku""",null,null,"""film/tri-orisky-pro-popelku/21…"
"""VOYO""",2013-07-07 13:20:00,"""Tři oříšky pro Popelku""",null,"""Film ČR/N (1973). Krásnou, ale…","""film/tri-orisky-pro-popelku/21…"
